In [1]:
! nvidia-smi

Sat Jun  4 08:09:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install pytorch-tabnet optuna 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 8.7 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 209 kB 58.6 MB/s 
     |████████████████████████████████| 78 kB 8.4 MB/s 
     |████████████████████████████████| 146 kB 64.8 MB/s 
     |████████████████████████████████| 112 kB 71.9 MB/s 
     |████████████████████████████████| 49 kB 7.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=ea49c6021bb2c1353a6a7dd839947936feda825bc0c22d1f2867d05d30dbcbb8
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score,classification_report
import optuna as opt
import torch
import os
import joblib

In [4]:
def make_save_cv_model(i,model_name,model,best_params,optim,output_path="./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/cross_validated_models"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))

In [5]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds=10,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    x = sc_df.drop([tar_col],axis=1)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = TabNetClassifier(n_d=trial.suggest_int("n_d", 8, 64),
                              n_a =trial.suggest_int("n_a", 8, 64),
                              n_steps = trial.suggest_int("n_steps",3,10),
                              gamma =trial.suggest_float("gamma", 1.0, 2.0),
                              n_independent = trial.suggest_int("n_independent",1,5),
                              n_shared = trial.suggest_int("n_shared",1,5),
                              momentum = trial.suggest_float("momentum", 0.01, 0.4),
                              optimizer_fn = torch.optim.Adam,
                              # scheduler_fn = torch.optim.lr_scheduler,
                              # scheduler_params = {"gamma" :trial.suggest_float("sch-gamma", 0.5, 0.95), "step_size": trial.suggest_int("sch_step_size", 10, 20, 2)},
                              verbose = verbose,
                              device_name = "auto"
                              )
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
      Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(Y_train.shape, Y_test.shape)
      clf.fit(X_train, Y_train,
              eval_set=[(X_test, Y_test)],
              eval_metric=['accuracy'])
      Y_pred = clf.predict(X_test)
      print(classification_report(Y_test, Y_pred, labels=[x for x in range(6)]))
      acc = accuracy_score(Y_pred, Y_test)
      return acc

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='maximize')
    study.optimize(objective, n_trials=optim_trial)
    best_params = study.best_params
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    joblib.dump(best_params,f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/comp/fold_{fold}_best_params.z")
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/fold_{fold}_best_params.txt", "w+") as file:file.write(str(best_params))
    print(f"Saved best_params at : outputs/{model_name}/best_params/fold_{fold}_best_params.txt")
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetClassifier(**study.best_params)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    clf_report = classification_report(Y_test, Y_pred, labels=[x for x in range(6)])
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/classification_report/{model_name}_{fold}_classification_report.txt","w+") as file:file.write(str(clf_report))
    accuracy = accuracy_score(Y_pred, Y_test)
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/{model_name}_{fold}_accuracy_score.txt","w+") as file:file.write(f" accuracy :: {str(accuracy)}")
    try:
        print("[++] Saving the model and parameters in corresponding directories")
        make_save_cv_model(fold,model_name,clf_model,best_params,optim=optim)
    except:
        print("[-] Failed to save the model")

In [6]:
use_df = pd.read_csv("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/data/trainable_scaled_balanced.csv")
tar_col = "PCE_categorical"
model_name = "pytorch_tabnet"
optimizer = "Adam"
fold_dict = joblib.load("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/inputs/fold_vals/fold_data.z")
fold = 6

In [7]:
train(fold_dict = fold_dict,
      fold = fold,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim=optimizer,
      optim_trial = 15)
print(f"[++] Ended the training process for fold {fold}")

[I 2022-06-04 08:10:14,798] A new study created in memory with name: no-name-4deb1fd2-5d1e-4e8d-bad9-c193d492b269


Starting optimization for fold : [6/10]
Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.62146 | val_0_accuracy: 0.51943 |  0:00:04s
epoch 1  | loss: 1.12912 | val_0_accuracy: 0.49784 |  0:00:08s
epoch 2  | loss: 1.1043  | val_0_accuracy: 0.56844 |  0:00:12s
epoch 3  | loss: 1.08207 | val_0_accuracy: 0.55181 |  0:00:17s
epoch 4  | loss: 1.11669 | val_0_accuracy: 0.56693 |  0:00:21s
epoch 5  | loss: 1.03224 | val_0_accuracy: 0.57275 |  0:00:25s
epoch 6  | loss: 0.96315 | val_0_accuracy: 0.60168 |  0:00:30s
epoch 7  | loss: 0.89346 | val_0_accuracy: 0.61118 |  0:00:34s
epoch 8  | loss: 0.82764 | val_0_accuracy: 0.62414 |  0:00:38s
epoch 9  | loss: 0.82639 | val_0_accuracy: 0.62953 |  0:00:43s
epoch 10 | loss: 0.79471 | val_0_accuracy: 0.64033 |  0:00:47s
epoch 11 | loss: 0.7704  | val_0_accuracy: 0.64465 |  0:00:51s
epoch 12 | loss: 0.77282 | val_0_accuracy: 0.65609 |  0:00:56s
epoch 13 | loss: 0.76677 | val_0_accuracy: 0.64767 |  0:01:00s
epoch 14 | loss: 0.7741  | val_0_accuracy:

[I 2022-06-04 08:12:37,970] Trial 0 finished with value: 0.6627806563039723 and parameters: {'n_d': 13, 'n_a': 56, 'n_steps': 8, 'gamma': 1.3568037572727158, 'n_independent': 2, 'n_shared': 3, 'momentum': 0.17395658802813507}. Best is trial 0 with value: 0.6627806563039723.


              precision    recall  f1-score   support

           0       0.97      1.00      0.99       826
           1       0.39      0.40      0.40       825
           2       0.63      0.72      0.67       822
           3       1.00      1.00      1.00       779
           4       0.40      0.24      0.30       761
           5       0.47      0.59      0.52       619

    accuracy                           0.66      4632
   macro avg       0.64      0.66      0.65      4632
weighted avg       0.65      0.66      0.65      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 3.52453 | val_0_accuracy: 0.40263 |  0:00:05s
epoch 1  | loss: 1.61691 | val_0_accuracy: 0.49007 |  0:00:11s
epoch 2  | loss: 1.2589  | val_0_accuracy: 0.47064 |  0:00:18s
epoch 3  | loss: 1.35691 | val_0_accuracy: 0.59737 |  0:00:24s
epoch 4  | loss: 0.90008 | val_0_accuracy: 0.63277 |  0:00:30s
epoch 5  | loss: 0.82197 | val_0_accuracy: 0.62478 |  0:00:36s
epoch 6  | loss: 0.77977 | val_0_accuracy: 

[I 2022-06-04 08:19:25,567] Trial 1 finished with value: 0.7789291882556131 and parameters: {'n_d': 55, 'n_a': 63, 'n_steps': 10, 'gamma': 1.471764426245155, 'n_independent': 3, 'n_shared': 3, 'momentum': 0.03428445332631497}. Best is trial 1 with value: 0.7789291882556131.


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.65      0.62      0.63       825
           2       0.83      0.95      0.89       822
           3       1.00      1.00      1.00       779
           4       0.58      0.55      0.57       761
           5       0.71      0.65      0.68       619

    accuracy                           0.80      4632
   macro avg       0.79      0.80      0.79      4632
weighted avg       0.80      0.80      0.80      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.13553 | val_0_accuracy: 0.63428 |  0:00:01s
epoch 1  | loss: 0.78969 | val_0_accuracy: 0.65652 |  0:00:03s
epoch 2  | loss: 0.73441 | val_0_accuracy: 0.66645 |  0:00:04s
epoch 3  | loss: 0.70626 | val_0_accuracy: 0.6794  |  0:00:06s
epoch 4  | loss: 0.6911  | val_0_accuracy: 0.69365 |  0:00:07s
epoch 5  | loss: 0.67334 | val_0_accuracy: 0.68631 |  0:00:09s
epoch 6  | loss: 0.66247 | val_0_accuracy: 

[I 2022-06-04 09:10:19,548] Trial 13 finished with value: 0.7886442141623489 and parameters: {'n_d': 45, 'n_a': 48, 'n_steps': 3, 'gamma': 1.9506646301997708, 'n_independent': 1, 'n_shared': 2, 'momentum': 0.3978009362141671}. Best is trial 12 with value: 0.8033246977547496.


              precision    recall  f1-score   support

           0       0.99      1.00      1.00       826
           1       0.63      0.60      0.62       825
           2       0.81      0.95      0.87       822
           3       1.00      1.00      1.00       779
           4       0.54      0.55      0.55       761
           5       0.70      0.57      0.63       619

    accuracy                           0.79      4632
   macro avg       0.78      0.78      0.78      4632
weighted avg       0.79      0.79      0.79      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.1195  | val_0_accuracy: 0.62284 |  0:00:02s
epoch 1  | loss: 0.80907 | val_0_accuracy: 0.64702 |  0:00:04s
epoch 2  | loss: 0.73373 | val_0_accuracy: 0.66818 |  0:00:06s
epoch 3  | loss: 0.70925 | val_0_accuracy: 0.64918 |  0:00:08s
epoch 4  | loss: 0.6941  | val_0_accuracy: 0.67142 |  0:00:10s
epoch 5  | loss: 0.69194 | val_0_accuracy: 0.67897 |  0:00:12s
epoch 6  | loss: 0.68081 | val_0_accuracy: 

[I 2022-06-04 09:12:06,362] Trial 14 finished with value: 0.7694300518134715 and parameters: {'n_d': 64, 'n_a': 50, 'n_steps': 3, 'gamma': 1.8231311695278785, 'n_independent': 3, 'n_shared': 2, 'momentum': 0.3867430535034889}. Best is trial 12 with value: 0.8033246977547496.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.59      0.56      0.57       825
           2       0.82      0.89      0.85       822
           3       1.00      1.00      1.00       779
           4       0.49      0.57      0.53       761
           5       0.70      0.54      0.61       619

    accuracy                           0.77      4632
   macro avg       0.77      0.76      0.76      4632
weighted avg       0.77      0.77      0.77      4632

 Best params for fold : [6/10]
{'n_d': 44, 'n_a': 51, 'n_steps': 3, 'gamma': 1.998898263952073, 'n_independent': 2, 'n_shared': 2, 'momentum': 0.36368393131019705}
Saved best_params at : outputs/pytorch_tabnet/best_params/fold_6_best_params.txt
Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.15244 |  0:00:01s
epoch 1  | loss: 0.83498 |  0:00:03s
epoch 2  | loss: 0.75363 |  0:00:05s
epoch 3  

Fold 0 has started running on 20-05-22 


Fold 0 has completed sucessfully on 17:00 20-05-22

Fold 1 has started running at 15:15 21-05-22

Fold 2 has started running at 09:45 22-05-22

Fold 2 has completed sucessfully on 10:58 22-05-22

Fold 3 has started running at 18:40 22-05-22

Fold 3 has completed sucessfully on 22-05-22

Fold 4 completed sucessfully on 21:04 on 22-05-22

Fold 5 started at 18:21 on 23-05-22

Fold 5 completed sucessfully on 19:44 on 23-05-22

Fold 6 started at 12:53 on 24-05-22

Fold 6 has completed at 14:14 on 24-05-22

Fold 7 started at 14:18 on 24-05-22

Fold 7 execution failed due to colab gpu time limit

Fold 7 trial 1 started at 11:00 on 25-05-22

Fold 7 has completed sucessfully at 12:14 on 25-05-22 

Fold 8 has started at 9:38 on 26-05-22

Fold 8 filed due to interrupted internet connection

Fold 8 trial 1 started at 13:38 on 26-05-22

Fold 8 has successfully executed at 15:33 on 26-05-22

Fold 9 has started at 13:35 on 27-05-22

Fold 9 has completed at 14:55 on 27-05-22

Editing with rectified dataset witout duplicacy because of space values

Fold 0 started at 13:21 on 28-05-22

Fold 0 completed sucessfully at 14:46 on 28-05-22

Fold 1 Failed to run due to some index error

_____ Restarting the training process again due to data distribution failure____



Fold 0 started at 10:47 on 30-05-22

Fold 0 completed successfully at 12:30 on 30-05-22

Fold 1 started at 8:27 on 31-05-22

Fold 1 execution failed due to runtime disconnection

Fold 1 started again at 9:38 on 31-05-22

Fold 1 execution failed due to gpu disconnect 

Fold 1 started again at 8:36 on 1-06-22

Fold 1 execution failed due to network disconnection

Fold 1 started again at 13:11 on 01-06-22

Fold 1 has succesfully executed at 14:25 on 01-06-22

Fold 2 started at 14:29 on 01-06-22

Fold 2 completed succesfully at 16:00 on 01-06-22

Fold 3 started at 09:43 on 02-06-22

Fold 3 execution failed due to gpu server disconnection 

Fold 3 started again at 13:34 on 02-06-22

Fold 3 ran successfully at 14:47 on 02-06-22

Fold 4 started at 14:48 on 02-06-22

Fold 4 ran successfully at 04:01 on 02-06-22

Fold 5 started at 13:51 on 03-06-22

Fold 5 ran successfully at 03:06 on 02-06-22

Fold 6 started at 15:11 on 03-06-22

Fold 6 stopped due to gpu run time limit

Fold 6 started again running at 13:40 on 04-06-22

Fold 6 completed at 14:45 on 04-06-22